In [ ]:
import numpy as np
import pandas as pd
import pickle

from modAL.models import ActiveLearner

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support
import xgboost as xgb


import recordlinkage as rl

from active_learn import*
from thresholding import*

from IPython import display
from matplotlib import pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
from importlib import reload
#reload(spam)

In [ ]:
#df_uns = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_uns_label_img_1.csv')
df_uns_red =pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_uns_label_img_red_hot.csv')

df_full = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\merged_clean_df.csv')
df_full.drop('Unnamed: 0',axis=1,inplace=True)

#Manually annotated data
test_1 = rl.read_annotation_file(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\result_bnb_trip.json')
test_2 = rl.read_annotation_file(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\result_bnb_vrb.json')

#Baseline with unsupervised
df_full_base = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_uns_label_base_train.csv')
df_full_base['label'] = df_full_base['label'].map(to_bin)

# Updated training df
#df_ams_train = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_ams_train_1.csv')

In [ ]:
# Divide test and training set for baseline 
df_base_train = df_full_base.loc[:1761] 
df_base_test = df_full_base.loc[1762:]

In [ ]:
compare(df_full,10161202, 10161074,full=False)

In [ ]:
#Labeled data
open_file = open(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\Double-Listing\Data\labels_no_hotels_v.pkl', "rb")
labels_test = pickle.load(open_file)
open_file.close()


In [ ]:
#Take out the labeled data for making a test set 
manual_labeled = []

for (tuple_1,tuple_2),label in labels_test:
    if label ==2 :
        manual_labeled.append([str((tuple_1,tuple_2)),np.float('nan')])
        manual_labeled.append([str((tuple_2,tuple_1)),np.float('nan')])
    else:    
        manual_labeled.append([str((tuple_1,tuple_2)),label])
        manual_labeled.append([str((tuple_2,tuple_1)),label])

 #Merging to have it with the full data   
df_labeled = pd.DataFrame(manual_labeled,columns=['ids','labels'])
df_labeled
with_labels =df_uns_red.merge(df_labeled,on='ids',how='left')


In [ ]:
#Comitee of models
def randomforest_com():
    clf = RandomForestClassifier()
    return clf
def desiciontree():
    clf= DecisionTreeClassifier()#criterion ='gini', max_depth= None,max_leaf_nodes= 7,min_samples_leaf= 3
    return clf
def logisticreg():
    clf = LogisticRegression()#fit_intercept= True,max_iter=150, penalty= 'l1',solver= 'saga'
    return clf
def xbg_class():
    clf = xgb.XGBClassifier() #learning_rate=0.1,max_depth= 7,n_estimators = 150
    return clf
def svm():
    clf = SVC(probability=True)#,C= 1,gamma= 0.1, kernel= 'linear'
    return clf
#Main model
def randomforest_main():
    clf = RandomForestClassifier(n_estimators=10,min_samples_split=2,warm_start=True,max_depth=5)
    return clf

list_models = [randomforest_com,desiciontree,logisticreg,xbg_class,svm,randomforest_main]

In [ ]:
users = ["Host_first_name_jaro_winker", "Host_first_name_levenshtein_sim" , "Host_first_name_jaccard_sim",
        "Host_first_name_relaxed_jaccard_sim", "Host_first_name_overlap_sim", "Host_first_name_containment_sim",
        'Host_picture_url_jaro_winker', 'Host_picture_url_levenshtein_sim',
        'Host_picture_url_jaccard_sim', 'Host_picture_url_relaxed_jaccard_sim',
        'Host_picture_url_overlap_sim', 'Host_picture_url_containment_sim']

df_ams_train =pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_ams_train_nh_1.csv')

In [ ]:
# Add labels to test set 
df_ams_test = df_uns_red.merge(df_labeled,on='ids',how='inner').dropna()
#Only run the rest of the code when starting from scratch otherwise just use updated df
#df_ams_train = with_labels[with_labels.labels.isna()]
df_ams_train.rename(columns={'labels':'label'},inplace=True)
df_ams_test.rename(columns={'labels':'label'},inplace=True)
#df_ams_train.drop(users,axis=1,inplace=True)
df_ams_test.drop(users,axis=1,inplace=True)


In [ ]:
#Recalculate the uns scores after taking out the test set
df_ams_train =df_ams_train.drop(['agg_score','weights','uns_label'],axis=1)
sorted_dataset = scoring_unsupervised(df_ams_train,baseline=False)
threshold,index = elbow_threshold(sorted_dataset) 
matches_score_weight, nonmatches_score_weight = unsupervised_labels(sorted_dataset,threshold)
columns = ['ids','agg_score','weights','uns_label']
uns_labels = matches_score_weight+nonmatches_score_weight
df_uns = pd.DataFrame(uns_labels,columns=columns)
df_ams_train= df_ams_train.merge(df_uns,on='ids')

In [ ]:
def al(df,df_test,queries = 20,baseline=False,committee_pred=False,incremental_comitee = False,boostrap = False,warm =False):

    if incremental_comitee == True:
        #Commitee of models
        members = [randomforest_main,desiciontree,logisticreg,xbg_class,svm]
    if incremental_comitee == False:
         members = [randomforest_com,desiciontree,logisticreg,xbg_class,svm]

    #List with comitee object
    learner_list = list()

    if baseline == False:
        #Only use the data corresponding to the features to train
        not_train = ['ListingId_1','ListingId_2','ids', 'agg_score', 'weights','uns_label','label']
    if baseline == True:
        #Only use the data corresponding to the features to train
        not_train = ['source_id','target_id','ids', 'agg_score', 'weights','uns_label','label']

    ids = df['ids'].values
    X_pool = df.drop(not_train, axis=1).values
    y_uns_label = df['uns_label'].values
    y_uns_weight = df['weights'].values

    y_pool_labels = df['label'].values
    y_test = df_test['label'].values
    X_test = df_test.drop(not_train, axis=1).values

    #Will save the new labels being labeled
    new_labels = []

    if boostrap == True:
        # Selecting the 2 most confident labels, a positive a negative with weight 1
        train_idx = np.where(y_uns_weight> 0.99998)
        #This is the labeled pool it starts with 2 examples
        X_train_lb = X_pool[train_idx]
        y_train_lb = y_uns_label[train_idx]
        #Delete used results
        X_pool = np.delete(X_pool, train_idx, axis=0)
        ids = np.delete(ids, train_idx, axis=0)
        y_uns_label = np.delete(y_uns_label, train_idx, axis=0)
        y_uns_weight = np.delete(y_uns_weight, train_idx, axis=0)
        y_pool_labels = np.delete(y_pool_labels, train_idx, axis=0)
    else:

        #If no boostrapping the model is initialized with random instances until there is one positive and negative
        train_idx = np.random.choice(X_pool.shape[0], 1, replace=False)
        X_train_lb = X_pool[train_idx]
        if baseline ==True:
                y_train_lb = y_pool_labels[train_idx]
        else:
            #
            if np.isnan(y_pool_labels[train_idx]):
                print("Are this two listings the same one? 1-match , 0-nonmatch, 2-more=info")
                ids_int = [int(x) for x in ids[train_idx][0][1:-1].split(',')]
                print(compare(df_full,ids_int[0],ids_int[1]))
                label = int(input())
                if label == 2:
                    print(compare(df_full,ids_int[0],ids_int[1],full=True))
                    label = int(input())
                    y_new = np.array([label], dtype=float)
                else:
                    y_new = np.array([label], dtype=float)
                new_labels.append([(ids_int[0],ids_int[1]),label])
                y_train_lb = y_new
            else:
                y_train_lb = y_pool_labels[train_idx]


        #Delete instances moved
        X_pool = np.delete(X_pool, train_idx, axis=0)
        ids = np.delete(ids, train_idx, axis=0)
        y_uns_label = np.delete(y_uns_label, train_idx, axis=0)
        y_uns_weight = np.delete(y_uns_weight, train_idx, axis=0)
        y_pool_labels = np.delete(y_pool_labels, train_idx, axis=0)


        while 1 not in y_train_lb or 0 not in y_train_lb :
            # Keep selecting till at least 1 postive and neagtive label has been added
            train_idx = np.random.choice(X_pool.shape[0], 1, replace=False)
            X_train_lb = np.concatenate((X_train_lb,X_pool[train_idx]),axis=0)
            #For baseline just take the correct label
            if baseline ==True:
                y_train_lb = np.concatenate((y_train_lb,y_pool_labels[train_idx]),axis =0)
            else:
                #If label not available ask for it to the oracle
                if np.isnan(y_pool_labels[train_idx]):
                    print("Are this two listings the same one? 1-match , 0-nonmatch, 2-more=info")
                    ids_int = [int(x) for x in ids[train_idx][0][1:-1].split(',')]
                    print(compare(df_full,ids_int[0],ids_int[1]))
                    label = int(input())
                    #Extra info needed
                    if label == 2:
                        print(compare(df_full,ids_int[0],ids_int[1],full=True))
                        label = int(input())
                        y_new = np.array([label], dtype=float)
                    else:
                        y_new = np.array([label], dtype=float)
                    new_labels.append([(ids_int[0],ids_int[1]),label])
                    y_train_lb = np.concatenate((y_train_lb,y_new),axis =0)
                #If label has already been given in past runs just take it
                else:
                    y_train_lb = np.concatenate((y_train_lb,y_pool_labels[train_idx]),axis =0)

            #Delete from unlabeled pool
            X_pool = np.delete(X_pool, train_idx, axis=0)
            ids = np.delete(ids, train_idx, axis=0)
            y_uns_label = np.delete(y_uns_label, train_idx, axis=0)
            y_uns_weight = np.delete(y_uns_weight, train_idx, axis=0)
            y_pool_labels = np.delete(y_pool_labels, train_idx, axis=0)

        #Check proggress
        print(X_train_lb.shape,y_train_lb.shape)

    #Only if warm adn boostraapp is specified we use the unsupervied labels
    if warm == True and boostrap == True:
        # initializing main random forest
        model_main = randomforest_main()
        model_main.fit(X_pool,y_uns_label,sample_weight= y_uns_weight)
        print('unsupervised')
    #Just Using  the warm true rf but not uns labels
    if warm == True and boostrap == False:
        # initializing main random forest
        model_main = randomforest_main()
        model_main.fit(X_train_lb,y_train_lb)
    #Normal RF
    if warm ==False:
        model_main = randomforest_com()
        model_main.fit(X_train_lb,y_train_lb)

    for clf in members:
        # initializing learner
        learner = ActiveLearner(
            estimator=clf(),
            X_training=X_train_lb, y_training=y_train_lb
            )
        learner_list.append(learner)


    # assembling the committee
    committee = Committee(learner_list=learner_list,query_strategy=vote_entropy_sampling,boots=boostrap)



    # we want to only use the prediction of the random forest which will be incrementally built

    #Get predictions of test set
    if committee_pred == False:
        y_pred = model_main.predict(X_test)
    if committee_pred == True:
        y_pred = committee.predict(X_test)

    #Calculate evaluation metrics
    precision_recall_fscore= precision_recall_fscore_support(y_test,y_pred,average='binary',zero_division=0)
    precision_scores = [precision_recall_fscore[0]]
    recall_scores=  [precision_recall_fscore[1]]
    f_score = [precision_recall_fscore[2]]



    # query by committee
    n_queries = queries
    #Active Learning Loop
    for idx in range(n_queries):
        print('Iteration:',idx)
        # Committee models gives the instance to be labeled
        query_idx, query_instance = committee.query(X_pool)

        if boostrap == True:
            #Get the predictions of the most informative instances
            preds =committee.predict(X_pool)
            #idx_new = np.array([np.argwhere(ids_main == ids[x])[0] for x in query_idx]).squeeze()

            #Only chose the instances which disagree with the unsupervised labels
            idx_reduced_bool = preds[query_idx] != y_uns_label[query_idx]

            if sum(idx_reduced_bool) > 1:
                #If there is more than one prediction disagreement chose the first one
                idx_reduced = query_idx[idx_reduced_bool][0]
                idx_reduced = np.expand_dims(idx_reduced, axis=0)

            else:
                #If there is not disgreement just chose the first instance which is gotten by vote entropy
                print('No disagreement uns labels and pred labels')
                idx_reduced = query_idx[0]
                idx_reduced = np.expand_dims(idx_reduced, axis=0)
        else:
            #If no boostrapping use the most informative instance by vote entropy only
            idx_reduced = query_idx[0]
            idx_reduced = np.expand_dims(idx_reduced, axis=0)


        #For AMS data we need to query the user directly
        if baseline == False:
            #Check if the label has already been given and saved
            if np.isnan(y_pool_labels[idx_reduced]):

                print("Are this two listings the same one? 1-match , 0-nonmatch, 2-more=info")
                ids_int = [int(x) for x in ids[idx_reduced][0][1:-1].split(',')]
                print(compare(df_full,ids_int[0],ids_int[1]))
                label = int(input())
                #If label 2 we need more info
                if label == 2:
                    print(compare(df_full,ids_int[0],ids_int[1],full=True))
                    label = int(input())
                    y_new = np.array([label], dtype=float)
                else:
                    y_new = np.array([label], dtype=float)
                new_labels.append([(ids_int[0],ids_int[1]),label])
            #If label is already given just take it and dont ask for it again, saving time hopefully
            else:
                ids_int = [int(x) for x in ids[idx_reduced][0][1:-1].split(',')]
                label = y_pool_labels[idx_reduced[0]]
                y_new = np.array([label], dtype=float)

        if baseline == True:
            #For baseline we can access the true labels no need to ask just retreive
            label = y_pool_labels[idx_reduced[0]]
            y_new = np.array([label], dtype=float)



        #This will add the labeled data into the same arrays
        X_train_lb = np.concatenate((X_train_lb,X_pool[idx_reduced]),axis=0)
        y_train_lb = np.concatenate((y_train_lb,y_new),axis =0)


        for model in committee:
            #Train the random forest in incremental way with the labeled instance
            if incremental_comitee == True:
                if type(model.get_params()['estimator'])== sklearn.ensemble._forest.RandomForestClassifier:

                    n_estimators = model.get_params()['estimator__n_estimators'] +2
                    params_rf = {'estimator__n_estimators':n_estimators} #,'estimator__max_depth': n_estimators
                    model.set_params(**params_rf)
                    #Teach the random forest , boostrap true is incremental
                    model.teach(X_train_lb, y_train_lb)

                    #Predict with the new instance
                    # we want to only use the prediction of the random forest which will be incrementally built
                else:

                    model.teach(X_train_lb, y_train_lb)
            else:
                model.teach(X_train_lb, y_train_lb)

        if warm == True and boostrap == True:
            #Teach the random forest Increase estimators for incremental learning
            model_main.n_estimators += 2
            #model_main.fit(X_pool_main,y_uns_label,sample_weight=y_uns_weight)
            model_main.fit(X_train_lb,y_train_lb,sample_weight =np.ones(y_train_lb.shape[0]))
        if warm == True and boostrap == False:
            #Teach the random forest Increase estimators for incremental learning
            model_main.n_estimators += 2
            #model_main.fit(X_pool_main,y_uns_label,sample_weight=y_uns_weight)
            model_main.fit(X_train_lb,y_train_lb)

        if warm ==False:
            model_main.fit(X_train_lb,y_train_lb)
        #Predict with the new instance
        # we want to only use the prediction of the random forest which will be incrementally built
        if committee_pred == False:
            y_pred = model_main.predict(X_test)
        else:
            y_pred = committee.predict(X_test)

        #Get evaluation scores
        precision_recall_fscore=precision_recall_fscore_support(y_test,y_pred,average='binary',zero_division=0)
        precision_scores.append(precision_recall_fscore[0])
        recall_scores.append(precision_recall_fscore[1])
        f_score.append(precision_recall_fscore[2])
        #Save results for further analysis
        dict_results = {'precision_scores':precision_scores,'recall_scores':recall_scores,
                        'f_score':f_score}

        #Delete the queried instance from the pool
        X_pool = np.delete(X_pool, idx_reduced, axis=0)
        ids = np.delete(ids, idx_reduced, axis=0)
        y_uns_label = np.delete(y_uns_label, idx_reduced, axis=0)
        y_pool_labels = np.delete(y_pool_labels, idx_reduced, axis=0)

        #Save labels to be added for further runs
        if baseline ==False:
            new_labels.append([(ids_int[0],ids_int[1]),label])


    #Graph to check the learning process
    with plt.style.context('seaborn-white'):
        plt.figure(figsize=(5, 5))
        plt.subplot(1, 1, 1)
        plt.title('Precision of your model')
        plt.plot(range(n_queries+1), precision_scores)
        #plt.scatter(range(n_queries+1), precision_scores)
        plt.xlabel('number of queries')
        plt.ylabel('Precision')
        display.display(plt.gcf())
        plt.close('all')
    with plt.style.context('seaborn-white'):
        plt.figure(figsize=(5, 5))
        plt.subplot(1, 1, 1)
        plt.title('recall_scores of your model')
        plt.plot(range(n_queries+1), recall_scores)
        #plt.scatter(range(n_queries+1), recall_scores)
        plt.xlabel('number of queries')
        plt.ylabel('recall_scores')
        display.display(plt.gcf())
        plt.close('all')
    with plt.style.context('seaborn-white'):
        plt.figure(figsize=(5, 5))
        plt.subplot(1, 1, 1)
        plt.title('f_score of your model')
        plt.plot(range(n_queries+1), f_score)
        #plt.scatter(range(n_queries+1), f_score)
        plt.xlabel('number of queries')
        plt.ylabel('f_score')
        display.display(plt.gcf())
        plt.close('all')
    return new_labels, dict_results, model_main

In [ ]:
multi = ['Xl_picture_url_img', 'LaBSE_emb_LaBSE_cos','Descrp_tfidf_tf_idf_cos', 'Lat_Lng_geo','Name_jaro_winker',
        'Host_thumbnail_url_jaro_winker', 'Host_thumbnail_url_levenshtein_sim', 'Host_thumbnail_url_jaccard_sim',
       'Host_thumbnail_url_relaxed_jaccard_sim','Host_thumbnail_url_overlap_sim', 'Host_thumbnail_url_containment_sim']
df_ams_train_no_multi = df_ams_train.drop(multi,axis=1)
df_ams_test_no_multi = df_ams_test.drop(multi,axis=1)

In [ ]:
df_ams_train_im = df_ams_train[['ListingId_1', 'ListingId_2','Xl_picture_url_img', 'ids', 'label', 'agg_score',
                               'weights', 'uns_label']]
df_ams_test_im = df_ams_test[['ListingId_1', 'ListingId_2','Xl_picture_url_img', 'ids', 'label', 'agg_score',
                               'weights', 'uns_label']]

In [ ]:
#Recalculate the uns scores after taking out the test set
df_ams_train_im =df_ams_train_im.drop(['agg_score','weights','uns_label'],axis=1)
sorted_dataset = scoring_unsupervised(df_ams_train_im,baseline=False)
threshold,index = elbow_threshold(sorted_dataset) 
matches_score_weight, nonmatches_score_weight = unsupervised_labels(sorted_dataset,threshold)
columns = ['ids','agg_score','weights','uns_label']
uns_labels = matches_score_weight+nonmatches_score_weight
df_uns = pd.DataFrame(uns_labels,columns=columns)
df_ams_train_im= df_ams_train_im.merge(df_uns,on='ids')

In [ ]:
img_results_full = []
for i in range(5):
    orcl_labels,im_results_full = al(df_ams_train_im,df_ams_test_im,queries=30,
                                            baseline=False,committee_pred= False,
                                            incremental_comitee = False,boostrap = True,
                                            warm= True)
    
    img_results_full.append(im_results_full)
    

In [ ]:
file_name = "img_results_full.pkl"

open_file = open(file_name, "wb")
pickle.dump(img_results_full, open_file)
open_file.close()

In [ ]:
df_ams_train_des = df_ams_train[['ListingId_1', 'ListingId_2','LaBSE_emb_LaBSE_cos','Descrp_tfidf_tf_idf_cos', 'ids', 'label', 'agg_score',
                               'weights', 'uns_label']]
df_ams_test_des = df_ams_test[['ListingId_1', 'ListingId_2','LaBSE_emb_LaBSE_cos','Descrp_tfidf_tf_idf_cos', 'ids', 'label', 'agg_score',
                               'weights', 'uns_label']]

In [ ]:
nlp_results_full = []
for i in range(5):
    orcl_labels,labse_results_full = al(df_ams_train_des,df_ams_test_des,queries=30,
                                            baseline=False,committee_pred= False,
                                            incremental_comitee = False,boostrap = True,
                                            warm= True)
    
    nlp_results_full.append(labse_results_full)

In [ ]:
file_name = "nlp_results_full.pkl"

open_file = open(file_name, "wb")
pickle.dump(nlp_results_full, open_file)
open_file.close()

In [ ]:
manual_labeled = []

for (tuple_1,tuple_2),label in orcl_labels:
    if label ==2 :
        manual_labeled.append([str((tuple_1,tuple_2)),np.float('nan')])
        manual_labeled.append([str((tuple_2,tuple_1)),np.float('nan')])
    else:    
        manual_labeled.append([str((tuple_1,tuple_2)),label])
        manual_labeled.append([str((tuple_2,tuple_1)),label])

#Save the labeled data into the main dataframe
for tuples,label in manual_labeled:
    df_ams_train['label'][df_ams_train['ids']== tuples] = label

In [ ]:
orcl_labels,no_modal_results_full = al(df_ams_train_no_multi,df_ams_test_no_multi,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = True,
                                        warm= True)

In [ ]:
orcl_labels,no_modal_results_full_1 = al(df_ams_train_no_multi,df_ams_test_no_multi,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = True,
                                        warm= True)


In [ ]:
orcl_labels,no_modal_results_full_2 = al(df_ams_train_no_multi,df_ams_test_no_multi,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = True,
                                        warm= True)

In [ ]:
orcl_labels,no_modal_results_full_3 = al(df_ams_train_no_multi,df_ams_test_no_multi,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = True,
                                        warm= True)

In [ ]:
orcl_labels,no_modal_results_full_4 = al(df_ams_train_no_multi,df_ams_test_no_multi,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = True,
                                        warm= True)

In [ ]:
orcl_labels,no_modal_results_full_5 = al(df_ams_train_no_multi,df_ams_test_no_multi,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = True,
                                        warm= True)

In [ ]:
no_modal_results_full = [no_modal_results_full,no_modal_results_full_1,no_modal_results_full_2,
                         no_modal_results_full_5,no_modal_results_full_4,no_modal_results_full_3]

file_name = "no_modal_results_full.pkl"

open_file = open(file_name, "wb")
pickle.dump(no_modal_results_full, open_file)
open_file.close()

In [ ]:
orcl_labels,ams_results_full = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = True,
                                        warm= True)

In [ ]:
#Take out the labeled data for making a test set 
manual_labeled = []

for (tuple_1,tuple_2),label in orcl_labels:
    if label ==2 :
        manual_labeled.append([str((tuple_1,tuple_2)),np.float('nan')])
        manual_labeled.append([str((tuple_2,tuple_1)),np.float('nan')])
    else:    
        manual_labeled.append([str((tuple_1,tuple_2)),label])
        manual_labeled.append([str((tuple_2,tuple_1)),label])

#Save the labeled data into the main dataframe
for tuples,label in manual_labeled:
    df_ams_train['label'][df_ams_train['ids']== tuples] = label

In [ ]:
orcl_labels,ams_results_full_1 = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = True,
                                        warm= True)

In [ ]:
#Take out the labeled data for making a test set 
manual_labeled = []

for (tuple_1,tuple_2),label in orcl_labels:
    if label ==2 :
        manual_labeled.append([str((tuple_1,tuple_2)),np.float('nan')])
        manual_labeled.append([str((tuple_2,tuple_1)),np.float('nan')])
    else:    
        manual_labeled.append([str((tuple_1,tuple_2)),label])
        manual_labeled.append([str((tuple_2,tuple_1)),label])

#Save the labeled data into the main dataframe
for tuples,label in manual_labeled:
    df_ams_train['label'][df_ams_train['ids']== tuples] = label

In [ ]:
orcl_labels,ams_results_full_2 = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = True,
                                        warm= True)

In [ ]:
#Take out the labeled data for making a test set 
manual_labeled = []

for (tuple_1,tuple_2),label in orcl_labels:
    if label ==2 :
        manual_labeled.append([str((tuple_1,tuple_2)),np.float('nan')])
        manual_labeled.append([str((tuple_2,tuple_1)),np.float('nan')])
    else:    
        manual_labeled.append([str((tuple_1,tuple_2)),label])
        manual_labeled.append([str((tuple_2,tuple_1)),label])

#Save the labeled data into the main dataframe
for tuples,label in manual_labeled:
    df_ams_train['label'][df_ams_train['ids']== tuples] = label

In [ ]:
orcl_labels,ams_results_full_3 = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = True,
                                        warm= True)

In [ ]:
#Take out the labeled data for making a test set 
manual_labeled = []

for (tuple_1,tuple_2),label in orcl_labels:
    if label ==2 :
        manual_labeled.append([str((tuple_1,tuple_2)),np.float('nan')])
        manual_labeled.append([str((tuple_2,tuple_1)),np.float('nan')])
    else:    
        manual_labeled.append([str((tuple_1,tuple_2)),label])
        manual_labeled.append([str((tuple_2,tuple_1)),label])

#Save the labeled data into the main dataframe
for tuples,label in manual_labeled:
    df_ams_train['label'][df_ams_train['ids']== tuples] = label

In [ ]:
orcl_labels,ams_results_full_4 = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = True,
                                        warm= True)

In [ ]:
#Take out the labeled data for making a test set 
manual_labeled = []

for (tuple_1,tuple_2),label in orcl_labels:
    if label ==2 :
        manual_labeled.append([str((tuple_1,tuple_2)),np.float('nan')])
        manual_labeled.append([str((tuple_2,tuple_1)),np.float('nan')])
    else:    
        manual_labeled.append([str((tuple_1,tuple_2)),label])
        manual_labeled.append([str((tuple_2,tuple_1)),label])

#Save the labeled data into the main dataframe
for tuples,label in manual_labeled:
    df_ams_train['label'][df_ams_train['ids']== tuples] = label

In [ ]:
resuts_full_ams = [ams_results_full,ams_results_full_1,ams_results_full_2,ams_results_full_3,ams_results_full_4]

file_name = "resuts_full_rf_ams_red.pkl"

open_file = open(file_name, "wb")
pickle.dump(resuts_full_ams, open_file)
open_file.close()

In [ ]:
#len(df_ams_train.label.dropna())
df_ams_train.to_csv('df_ams_train_nh_1.csv',index=False)

In [ ]:
orcl_labels,ams_results_main_no_boot = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = False,
                                        warm= True)

In [ ]:
orcl_labels,ams_results_main_no_boot_1 = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = False,
                                        warm= True)

In [ ]:
orcl_labels,ams_results_main_no_boot_2 = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = False,
                                        warm= True)

In [ ]:
orcl_labels,ams_results_main_no_boot_3 = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = False,
                                        warm= True)

In [ ]:
orcl_labels,ams_results_main_no_boot_4 = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = False,
                                        warm= True)

In [ ]:
resuts_noboot_ams = [ams_results_main_no_boot_4,ams_results_main_no_boot_3,ams_results_main_no_boot_2
                     ,ams_results_main_no_boot_1,ams_results_main_no_boot]

file_name = "resuts_noboot_ams.pkl"

open_file = open(file_name, "wb")
pickle.dump(resuts_noboot_ams, open_file)
open_file.close()

In [ ]:
#Take out the labeled data for making a test set 
manual_labeled = []

for (tuple_1,tuple_2),label in orcl_labels:
    if label ==2 :
        manual_labeled.append([str((tuple_1,tuple_2)),np.float('nan')])
        manual_labeled.append([str((tuple_2,tuple_1)),np.float('nan')])
    else:    
        manual_labeled.append([str((tuple_1,tuple_2)),label])
        manual_labeled.append([str((tuple_2,tuple_1)),label])

#Save the labeled data into the main dataframe
for tuples,label in manual_labeled:
    df_ams_train['label'][df_ams_train['ids']== tuples] = label

In [ ]:
orcl_labels,ams_results_main_no_boot_no_warm = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = False,
                                        warm= False)

In [ ]:
orcl_labels,ams_results_main_no_boot_no_warm_1 = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = False,
                                        warm= False)

In [ ]:
orcl_labels,ams_results_main_no_boot_no_warm_2 = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = False,
                                        warm= False)

In [ ]:
orcl_labels,ams_results_main_no_boot_no_warm_3 = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = False,
                                        warm= False)

In [ ]:
orcl_labels,ams_results_main_no_boot_no_warm_4 = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = False,
                                        warm= False)

In [ ]:
#Take out the labeled data for making a test set 
manual_labeled = []

for (tuple_1,tuple_2),label in orcl_labels:
    if label ==2 :
        manual_labeled.append([str((tuple_1,tuple_2)),np.float('nan')])
        manual_labeled.append([str((tuple_2,tuple_1)),np.float('nan')])
    else:    
        manual_labeled.append([str((tuple_1,tuple_2)),label])
        manual_labeled.append([str((tuple_2,tuple_1)),label])

#Save the labeled data into the main dataframe
for tuples,label in manual_labeled:
    df_ams_train['label'][df_ams_train['ids']== tuples] = label
df_ams_train.to_csv('df_ams_train_nh_1.csv',index=False)

In [ ]:
resuts_no_boot_no_warm_ams = [ams_results_main_no_boot_no_warm_4,ams_results_main_no_boot_no_warm_3,ams_results_main_no_boot_no_warm_2
                     ,ams_results_main_no_boot_no_warm_1,ams_results_main_no_boot_no_warm]

file_name = "resuts_no_boot_no_warm_ams.pkl"

open_file = open(file_name, "wb")
pickle.dump(resuts_no_boot_no_warm_ams, open_file)
open_file.close()

In [ ]:
#Create an average of the 5 runs of the active learning set up 
precision_full_rf = []
recall_full_rf = []
f_score_full_rf = []

precision_no_boot_rf = []
recall_no_boot_rf = []
f_score_no_boot_rf = []

precision_rf_noboot_nowarm = []
recall_rf_noboot_nowarm = []
f_score_rf_noboot_nowarm = []

for dicts in resuts_full_ams:
    precision_full_rf.append(dicts['precision_scores'])
    recall_full_rf.append(dicts['recall_scores'])
    f_score_full_rf.append(dicts['f_score'])

precision_full_rf_mean =np.mean(precision_full_rf,axis=0)
recall_full_rf_mean =np.mean(recall_full_rf,axis=0)
f_score_full_rf_mean = np.mean(f_score_full_rf,axis=0)

precision_full_rf_sd =np.std(precision_full_rf,axis=0)
recall_full_rf_sd =np.std(recall_full_rf,axis=0)
f_score_full_rf_sd = np.std(f_score_full_rf,axis=0)


for dicts in resuts_noboot_ams:
    precision_no_boot_rf.append(dicts['precision_scores'])
    recall_no_boot_rf.append(dicts['recall_scores'])
    f_score_no_boot_rf.append(dicts['f_score'])

precision_no_boot_rf_mean =np.mean(precision_no_boot_rf,axis=0)
recall_no_boot_rf_mean =np.mean(recall_no_boot_rf,axis=0)
f_score_no_boot_rf_mean = np.mean(f_score_no_boot_rf,axis=0)

precision_no_boot_rf_sd =np.std(precision_no_boot_rf,axis=0)
recall_no_boot_rf_sd =np.std(recall_no_boot_rf,axis=0)
f_score_no_boot_rf_sd = np.std(f_score_no_boot_rf,axis=0)

for dicts in resuts_no_boot_no_warm_ams:
    precision_rf_noboot_nowarm.append(dicts['precision_scores'])
    recall_rf_noboot_nowarm.append(dicts['recall_scores'])
    f_score_rf_noboot_nowarm.append(dicts['f_score'])

precision_rf_noboot_nowarm_mean =np.mean(precision_rf_noboot_nowarm,axis=0)
recall_rf_noboot_nowarm_mean =np.mean(recall_rf_noboot_nowarm,axis=0)
f_score_rf_noboot_nowarm_mean = np.mean(f_score_rf_noboot_nowarm,axis=0)

precision_rf_noboot_nowarm_sd =np.std(precision_rf_noboot_nowarm,axis=0)
recall_rf_noboot_nowarm_mean =np.std(recall_rf_noboot_nowarm,axis=0)
f_score_rf_noboot_nowarm_sd = np.std(f_score_rf_noboot_nowarm,axis=0)

In [ ]:

fig, ax = plt.subplots(1)
fig.set_size_inches(10,8)
ax.plot(np.arange(31), f_score_full_rf_mean, lw=1, label='boot rf', color='blue')

ax.plot(np.arange(31), f_score_no_boot_rf_mean, lw=1, label=' no boot rf', color='green')
ax.plot(np.arange(31), f_score_rf_noboot_nowarm_mean, lw=1, label='no boot no warm rf', color='purple')
#ax.plot([0, 31], [0.7959183673469387,0.7959183673469387], 'k-', lw=2)
ax.fill_between(np.arange(31), f_score_full_rf_mean + f_score_full_rf_sd, f_score_full_rf_mean-f_score_full_rf_sd
                ,facecolor='blue', alpha=0.2)

ax.fill_between(np.arange(31), f_score_no_boot_rf_mean + f_score_no_boot_rf_sd, f_score_no_boot_rf_mean-f_score_no_boot_rf_sd
                ,facecolor='green', alpha=0.1)
ax.fill_between(np.arange(31), f_score_rf_noboot_nowarm_mean + f_score_rf_noboot_nowarm_sd, f_score_rf_noboot_nowarm_mean-f_score_rf_noboot_nowarm_sd
                ,facecolor='purple', alpha=0.1)                                
ax.set_title('F score')
ax.legend(loc='lower right')
ax.set_xlabel('Iterations')
ax.set_ylabel('F_score')

ax.grid()

In [ ]:
_,dict_results_com = al(df_base_train,df_test = df_base_test ,queries=1,
                                        baseline=True,committee_pred= True,
                                        incremental_comitee = True,boostrap = True,
                                        warm= True)

In [ ]:
resuts_full_comitte = []
for i in range(5):    
    _,dict_results_com = al(df_base_train,df_test = df_base_test ,queries=100,
                                        baseline=True,committee_pred= True,
                                        incremental_comitee = True,boostrap = True,
                                        warm= True)
    resuts_full_comitte.append(dict_results_com)


In [ ]:
file_name = "resuts_full_comitte.pkl"

open_file = open(file_name, "wb")
pickle.dump(resuts_full_comitte, open_file)
open_file.close()

In [ ]:
resuts_full_rf = []
for i in range(5):   
    _,dict_results_main = al(df_base_train,df_test = df_base_test ,queries=100,
                                        baseline=True,committee_pred= False,
                                        incremental_comitee = False,boostrap = True,
                                        warm=True)
    resuts_full_rf.append(dict_results_main)


In [ ]:
file_name = "resuts_full_rf.pkl"

open_file = open(file_name, "wb")
pickle.dump(resuts_full_rf, open_file)
open_file.close()

In [ ]:
resuts_rf_no_boot = []
for i in range(5): 
    _,dict_results_main_no_boot = al(df_base_train,df_test = df_base_test ,queries=100,
                                        baseline=True,committee_pred= False,
                                        incremental_comitee = False,boostrap = False,
                                        warm=True)
    resuts_rf_no_boot.append(dict_results_main_no_boot)

In [ ]:
file_name = "resuts_rf_no_boot.pkl"

open_file = open(file_name, "wb")
pickle.dump(resuts_rf_no_boot, open_file)
open_file.close()

In [ ]:
resuts_rf_noboot_nowarm = []
for i in range(5):
    _,dict_results_main_no_boot_no_warm = al(df_base_train,df_test = df_base_test ,queries=100,
                                        baseline=True,committee_pred= False,
                                        incremental_comitee = False,boostrap=False,
                                        warm=False)
    resuts_rf_noboot_nowarm.append(dict_results_main_no_boot_no_warm)

In [ ]:
file_name = "resuts_rf_noboot_nowarm.pkl"

open_file = open(file_name, "wb")
pickle.dump(resuts_rf_noboot_nowarm, open_file)
open_file.close()

In [ ]:
file_name = 'resuts_full_rf.pkl'
open_file = open(file_name, "rb")
resuts_full_rf = pickle.load(open_file)
open_file.close()

file_name = 'resuts_full_comitte.pkl'
open_file = open(file_name, "rb")
resuts_full_comitte = pickle.load(open_file)
open_file.close()

file_name = 'resuts_rf_no_boot.pkl'
open_file = open(file_name, "rb")
resuts_rf_no_boot = pickle.load(open_file)
open_file.close()

file_name = 'resuts_rf_noboot_nowarm.pkl'
open_file = open(file_name, "rb")
resuts_noboot_nowarm = pickle.load(open_file)
open_file.close()

In [ ]:
#Create an average of the 5 runs of the active learning set up 
precision_full_rf = []
recall_full_rf = []
f_score_full_rf = []

precision_full_cmt = []
recall_full_cmt = []
f_score_full_cmt = []

precision_no_boot_rf = []
recall_no_boot_rf = []
f_score_no_boot_rf = []

precision_rf_noboot_nowarm = []
recall_rf_noboot_nowarm = []
f_score_rf_noboot_nowarm = []

for dicts in resuts_full_rf:
    precision_full_rf.append(dicts['precision_scores'])
    recall_full_rf.append(dicts['recall_scores'])
    f_score_full_rf.append(dicts['f_score'])

precision_full_rf_mean =np.mean(precision_full_rf,axis=0)
recall_full_rf_mean =np.mean(recall_full_rf,axis=0)
f_score_full_rf_mean = np.mean(f_score_full_rf,axis=0)

precision_full_rf_sd =np.std(precision_full_rf,axis=0)
recall_full_rf_sd =np.std(recall_full_rf,axis=0)
f_score_full_rf_sd = np.std(f_score_full_rf,axis=0)

for dicts in resuts_full_comitte:
    precision_full_cmt.append(dicts['precision_scores'])
    recall_full_cmt.append(dicts['recall_scores'])
    f_score_full_cmt.append(dicts['f_score'])

precision_full_cmt_mean =np.mean(precision_full_cmt,axis=0)
recall_full_cmt_mean =np.mean(recall_full_cmt,axis=0)
f_score_full_cmt_mean = np.mean(f_score_full_cmt,axis=0)

precision_full_cmt_sd =np.std(precision_full_cmt,axis=0)
recall_full_cmt_sd =np.std(recall_full_cmt,axis=0)
f_score_full_cmt_sd = np.std(f_score_full_cmt,axis=0)

for dicts in resuts_rf_no_boot:
    precision_no_boot_rf.append(dicts['precision_scores'])
    recall_no_boot_rf.append(dicts['recall_scores'])
    f_score_no_boot_rf.append(dicts['f_score'])

precision_no_boot_rf_mean =np.mean(precision_no_boot_rf,axis=0)
recall_no_boot_rf_mean =np.mean(recall_no_boot_rf,axis=0)
f_score_no_boot_rf_mean = np.mean(f_score_no_boot_rf,axis=0)

precision_no_boot_rf_sd =np.std(precision_no_boot_rf,axis=0)
recall_no_boot_rf_sd =np.std(recall_no_boot_rf,axis=0)
f_score_no_boot_rf_sd = np.std(f_score_no_boot_rf,axis=0)

for dicts in resuts_rf_noboot_nowarm:
    precision_rf_noboot_nowarm.append(dicts['precision_scores'])
    recall_rf_noboot_nowarm.append(dicts['recall_scores'])
    f_score_rf_noboot_nowarm.append(dicts['f_score'])

precision_rf_noboot_nowarm_mean =np.mean(precision_rf_noboot_nowarm,axis=0)
recall_rf_noboot_nowarm_mean =np.mean(recall_rf_noboot_nowarm,axis=0)
f_score_rf_noboot_nowarm_mean = np.mean(f_score_rf_noboot_nowarm,axis=0)

precision_rf_noboot_nowarm_sd =np.std(precision_rf_noboot_nowarm,axis=0)
recall_rf_noboot_nowarm_mean =np.std(recall_rf_noboot_nowarm,axis=0)
f_score_rf_noboot_nowarm_sd = np.std(f_score_rf_noboot_nowarm,axis=0)

In [ ]:
from matplotlib.pyplot import figure
fig, ax = plt.subplots(1)
fig.set_size_inches(10,8)
ax.plot(np.arange(101), f_score_full_rf_mean, lw=1, label='boot rf', color='blue')
#ax.plot(np.arange(101), f_score_full_cmt_mean, lw=1, label='boot committee', color='red')
ax.plot(np.arange(101), f_score_no_boot_rf_mean, lw=1, label=' no boot rf', color='green')
ax.plot(np.arange(101), f_score_rf_noboot_nowarm_mean, lw=1, label='no boot no warm rf', color='purple')
ax.plot([0, 101], [0.7959183673469387,0.7959183673469387], 'k-', lw=2)
ax.fill_between(np.arange(101), f_score_full_rf_mean + f_score_full_rf_sd, f_score_full_rf_mean-f_score_full_rf_sd
                ,facecolor='blue', alpha=0.2)
#ax.fill_between(np.arange(101), f_score_full_cmt_mean + f_score_full_cmt_sd, f_score_full_cmt_mean-f_score_full_cmt_sd
#                ,facecolor='red', alpha=0.2)
ax.fill_between(np.arange(101), f_score_no_boot_rf_mean + f_score_no_boot_rf_sd, f_score_no_boot_rf_mean-f_score_no_boot_rf_sd
                ,facecolor='green', alpha=0.1)
ax.fill_between(np.arange(101), f_score_rf_noboot_nowarm_mean + f_score_rf_noboot_nowarm_sd, f_score_rf_noboot_nowarm_mean-f_score_rf_noboot_nowarm_sd
                ,facecolor='purple', alpha=0.1)                                
ax.set_title('F score')
ax.legend(loc='lower right')
ax.set_xlabel('Iterations')
ax.set_ylabel('F_score')

ax.grid()

In [ ]:
#Labeled data
open_file = open('labels-all.pkl', "rb")
labels_test = pickle.load(open_file)
open_file.close()

manual_labeled = []

for (tuple_1,tuple_2),label in labels_test:
    if label ==2 :
        manual_labeled.append([str((tuple_1,tuple_2)),np.float('nan')])
        manual_labeled.append([str((tuple_2,tuple_1)),np.float('nan')])
    else:    
        manual_labeled.append([str((tuple_1,tuple_2)),label])
        manual_labeled.append([str((tuple_2,tuple_1)),label])

 #Merging to have it with the full data   
df_labeled = pd.DataFrame(manual_labeled,columns=['ids','labels'])
df_labeled[df_labeled['labels']==1]

In [ ]:
compare(df_full,10908624, 10908624,full=True)

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.concat([df_full[df_full['ListingId']==19184032],df_full[df_full['ListingId']==12058561]]).transpose().dropna()

In [ ]:
x =33359533
#compare(df_full,18113908, 10175956,full=True)
12058561
compare(df_full,19184032,12058561,full=True)

In [ ]:
df_uns[(df_uns['ListingId_1']==19184032) & (df_uns['ListingId_2']==12058561)].transpose()

In [ ]:
df_uns[df_uns['Xl_picture_url_img']==1]

In [ ]:
_,_,model = al(df_ams_train,df_ams_test,queries=30,
                                        baseline=False,committee_pred= False,
                                        incremental_comitee = False,boostrap = True,
                                        warm= True)

In [ ]:
not_train = ['ListingId_1','ListingId_2','ids', 'agg_score', 'weights','uns_label','label']
X_pool = df_ams_train.drop(not_train, axis=1).values
labels_pred = model.predict(X_pool)
df_ams_train['pred_label'] = labels_pred

In [ ]:
df_final = df_ams_train[['ListingId_1','ListingId_2','ids','uns_label','label','pred_label']]
df_final.to_csv('df_final.csv')